In [ ]:
# The following can be run to transform UCI DSA dataset to numpy array

import numpy as np
import os
from glob import glob

data_dir = './data'

domain_names = ['T', 'RA', 'LA', 'RL', 'LL']
domain_indices = {
    'T': list(range(0, 9)),
    'RA': list(range(9, 18)),
    'LA': list(range(18, 27)),
    'RL': list(range(27, 36)),
    'LL': list(range(36, 45)),
}

train_x, train_y = [], []
test_x, test_y = [], []

for activity_idx in range(1, 20):  # a01–a19
    activity_folder = f"a{activity_idx:02d}"
    for person_idx in range(1, 9):  # p1–p8
        person_folder = f"p{person_idx}"
        segment_path = os.path.join(data_dir, activity_folder, person_folder, "*.txt")
        segment_files = sorted(glob(segment_path))

        for filepath in segment_files:
            try:
                data = np.loadtxt(filepath, delimiter=",")  # shape: (125, 45)
                if data.shape != (125, 45):
                    continue

                # Extract each domain (125, 9), then stack → (125, 9, 5)
                domain_stack = np.stack(
                    [data[:, domain_indices[d]] for d in domain_names], axis=-1
                )


                # Reshape to (125*9, 5) = (1125, 5)
                reshaped = domain_stack.reshape(125 * 9, 5)

                if person_idx <= 6:
                    train_x.append(reshaped)
                    train_y.append(activity_idx - 1)
                else:
                    test_x.append(reshaped)
                    test_y.append(activity_idx - 1)

            except Exception as e:
                print(f"Error loading {filepath}: {e}")

# Convert to numpy arrays
train_x = np.array(train_x)  # shape: (6840, 1125, 5)
train_y = np.array(train_y)  # shape: (6840,)
test_x = np.array(test_x)    # shape: (2280, 1125, 5)
test_y = np.array(test_y)    # shape: (2280,)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

np.save("data_train.npy", train_x)
np.save("data_test.npy", test_x)
np.save("target_train.npy", train_y)
np.save("target_test.npy", test_y)


